In [1]:
import keras

Using TensorFlow backend.


In [2]:
import pandas
from sklearn.utils import shuffle
import gensim
import numpy as np

In [3]:
raw_data = pandas.DataFrame.from_csv(path='../data/people_attributes.csv', index_col=None)
raw_data = raw_data.set_index('name', drop=False)
raw_data.head()

,name,occupation,description,gender
name,,,,
Barack Obama,Barack Obama,Politician,44th President of the United States of America,male
Barbara Boxer,Barbara Boxer,Politician,American politician,female
Bill Clinton,Bill Clinton,Politician,42nd President of the United States,male
Carly Fiorina,Carly Fiorina,Politician,American corporate executive and politician,female
Clint Eastwood,Clint Eastwood,Actor,actor and director from the United States,male


In [4]:
raw_data['gender'].replace('female', 0, inplace=True)
raw_data['gender'].replace('male', 1, inplace=True)
raw_data.head()

,name,occupation,description,gender
name,,,,
Barack Obama,Barack Obama,Politician,44th President of the United States of America,1
Barbara Boxer,Barbara Boxer,Politician,American politician,0
Bill Clinton,Bill Clinton,Politician,42nd President of the United States,1
Carly Fiorina,Carly Fiorina,Politician,American corporate executive and politician,0
Clint Eastwood,Clint Eastwood,Actor,actor and director from the United States,1


In [5]:
name_and_gender = raw_data['gender']
name_and_gender = shuffle(name_and_gender)
name_and_gender.head()

name
Maria Cantwell      0
Kevin Johnson       1
Marissa Mayer       0
Kelly Ayotte        0
Kushal Pal Singh    1
Name: gender, dtype: int64

In [6]:
name_and_gender.head()

name
Maria Cantwell      0
Kevin Johnson       1
Marissa Mayer       0
Kelly Ayotte        0
Kushal Pal Singh    1
Name: gender, dtype: int64

In [7]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin', binary=True) 

In [9]:
name_and_gender['Barack Obama']

1

In [10]:
def _name_not_has_vec(name):
    try:
        word2vec.word_vec(name.replace(' ','_'))
        return False
    except:
        return True

In [11]:
truncated_name_and_gender = name_and_gender.drop([name for name in name_and_gender.keys() if _name_not_has_vec(name)])

In [12]:
truncated_name_and_gender.describe()

count    241.000000
mean       0.804979
std        0.397041
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: gender, dtype: float64

In [13]:
truncated_name_and_gender.head()

name
Maria Cantwell      0
Marissa Mayer       0
Kelly Ayotte        0
Kushal Pal Singh    1
Satya Prabhakar     1
Name: gender, dtype: int64

In [14]:
name_and_vector = pandas.DataFrame(truncated_name_and_gender.keys(), columns=['name'])
name_and_vector.head()

,name
0,Maria Cantwell
1,Marissa Mayer
2,Kelly Ayotte
3,Kushal Pal Singh
4,Satya Prabhakar


In [84]:
def _get_vector(row):
    return word2vec.word_vec(row.replace(' ','_')).flatten()

In [85]:
vectors = name_and_vector.name.map(_get_vector)
vectors = vectors.apply(pandas.Series)

In [86]:
vectors.values

array([[ 0.00939941, -0.1484375 , -0.20117188, ..., -0.21484375,
         0.15234375,  0.25585938],
       [-0.01257324,  0.24902344, -0.34570312, ..., -0.42382812,
         0.37890625, -0.00549316],
       [ 0.14648438,  0.02954102, -0.13769531, ...,  0.08056641,
        -0.18359375,  0.00263977],
       ..., 
       [ 0.21582031,  0.32226562, -0.09863281, ..., -0.01831055,
        -0.54296875, -0.22363281],
       [-0.13867188, -0.23046875, -0.07519531, ...,  0.20507812,
        -0.19238281,  0.08105469],
       [-0.13964844,  0.07128906, -0.38476562, ..., -0.15429688,
        -0.01049805, -0.37304688]], dtype=float32)

In [35]:
just_binary_genders = np.array(truncated_name_and_gender)
just_binary_genders

array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [92]:
train_data = vectors[:200].values
train_labels = just_binary_genders[:200]
test_data = vectors[200:].values
test_labels = just_binary_genders[200:]
train_data.shape

(200, 300)

In [108]:
model = keras.models.Sequential([
    keras.layers.Dense(1, input_shape=(300,), activation='sigmoid'),
    ])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 1)                 301       
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [109]:
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

In [110]:
model.fit(train_data, train_labels, verbose=2, epochs=100, validation_data=(test_data, test_labels))

Train on 200 samples, validate on 41 samples
Epoch 1/100
0s - loss: 0.2577 - acc: 0.4650 - val_loss: 0.2279 - val_acc: 0.6098
Epoch 2/100
0s - loss: 0.2332 - acc: 0.6700 - val_loss: 0.2058 - val_acc: 0.8293
Epoch 3/100
0s - loss: 0.2134 - acc: 0.7450 - val_loss: 0.1874 - val_acc: 0.7805
Epoch 4/100
0s - loss: 0.1961 - acc: 0.7750 - val_loss: 0.1746 - val_acc: 0.8293
Epoch 5/100
0s - loss: 0.1845 - acc: 0.7850 - val_loss: 0.1636 - val_acc: 0.8293
Epoch 6/100
0s - loss: 0.1736 - acc: 0.7950 - val_loss: 0.1551 - val_acc: 0.8537
Epoch 7/100
0s - loss: 0.1655 - acc: 0.7950 - val_loss: 0.1479 - val_acc: 0.8537
Epoch 8/100
0s - loss: 0.1585 - acc: 0.7950 - val_loss: 0.1419 - val_acc: 0.8537
Epoch 9/100
0s - loss: 0.1528 - acc: 0.7950 - val_loss: 0.1371 - val_acc: 0.8537
Epoch 10/100
0s - loss: 0.1481 - acc: 0.7950 - val_loss: 0.1330 - val_acc: 0.8537
Epoch 11/100
0s - loss: 0.1441 - acc: 0.7950 - val_loss: 0.1298 - val_acc: 0.8537
Epoch 12/100
0s - loss: 0.1406 - acc: 0.7950 - val_loss: 0.127

In [132]:
test_predict = model.predict(test_data)
test_predict

array([[ 0.82870197],
       [ 0.77971435],
       [ 0.87402689],
       [ 0.97314125],
       [ 0.74221998],
       [ 0.80812418],
       [ 0.8621847 ],
       [ 0.9106319 ],
       [ 0.64999831],
       [ 0.44281057],
       [ 0.87099701],
       [ 0.88975173],
       [ 0.85612589],
       [ 0.97617835],
       [ 0.96385831],
       [ 0.78015208],
       [ 0.91499949],
       [ 0.97594589],
       [ 0.24922973],
       [ 0.81384611],
       [ 0.91863841],
       [ 0.66546541],
       [ 0.35794035],
       [ 0.72538179],
       [ 0.71342397],
       [ 0.91353798],
       [ 0.4794949 ],
       [ 0.81853193],
       [ 0.7869367 ],
       [ 0.92711145],
       [ 0.94755   ],
       [ 0.42940682],
       [ 0.89408821],
       [ 0.77572149],
       [ 0.82696468],
       [ 0.98620403],
       [ 0.62208581],
       [ 0.822106  ],
       [ 0.98360461],
       [ 0.94520593],
       [ 0.94627374]], dtype=float32)

In [113]:
test_labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [114]:
test_names = truncated_name_and_gender[200:]
test_names

name
Bharat Desai          1
Chris Coons           1
Wilfred Kiboro        1
John P. Surma         1
Sheldon Whitehouse    1
Tom Udall             1
Harry Reid            1
Jeb Bush              1
David Vitter          1
Susana Martinez       0
Gregory Wasson        1
Joe Manchin           1
Carol Meyrowitz       0
Anil Ambani           1
Gregg Steinhafel      1
Doug Ducey            1
Richard Fairbank      1
Jeff Bezos            1
Anne Hathaway         0
Rajeev Suri           1
John Kasich           1
Tatsumi Kimishima     1
Margot Robbie         0
Bill Clinton          1
Cory Booker           1
Jesse Eisenberg       1
Ben Cardin            1
Tom Cruise            1
Martin O'Malley       1
Robert Redford        1
John Thain            1
Meg Whitman           0
Thom Tillis           1
Butch Otter           1
Sergio Ermotti        1
Dieter Zetsche        1
Debbie Stabenow       0
Devin Wenig           1
Elon Musk             1
Bernie Sanders        1
Bob Menendez          1
Name: gende

In [135]:
test_names.shape
test_predict_extracted = []
for i in test_predict:
    test_predict_extracted.append(i[0])
test_predict_extracted
#test_predict = pandas.Series(test_predict[0])

[0.82870197,
 0.77971435,
 0.87402689,
 0.97314125,
 0.74221998,
 0.80812418,
 0.8621847,
 0.9106319,
 0.64999831,
 0.44281057,
 0.87099701,
 0.88975173,
 0.85612589,
 0.97617835,
 0.96385831,
 0.78015208,
 0.91499949,
 0.97594589,
 0.24922973,
 0.81384611,
 0.91863841,
 0.66546541,
 0.35794035,
 0.72538179,
 0.71342397,
 0.91353798,
 0.4794949,
 0.81853193,
 0.7869367,
 0.92711145,
 0.94755,
 0.42940682,
 0.89408821,
 0.77572149,
 0.82696468,
 0.98620403,
 0.62208581,
 0.822106,
 0.98360461,
 0.94520593,
 0.94627374]

In [142]:
test_predict_extracted = pandas.DataFrame(test_predict_extracted)

In [143]:
test_predict_extracted

,0
0,0.828702
1,0.779714
2,0.874027
3,0.973141
4,0.742220
5,0.808124
6,0.862185
7,0.910632
8,0.649998
9,0.442811


In [145]:
test_names_predictions = pandas.DataFrame(test_names)
test_names_predictions = test_names_predictions.map(test_predict_extracted)

AttributeError: 'DataFrame' object has no attribute 'map'